<a href="https://colab.research.google.com/github/mausamsion/playing_with_Sequences/blob/master/Disecting_LSTMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference:
https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html

Actual paper:
https://arxiv.org/abs/1409.0473

# What the heck is LSTM/GRU actually doing !!!

In [0]:
!python -m spacy download en
!python -m spacy download de
!pip install --upgrade torchtext

In [0]:
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

SRC = Field(tokenize = "spacy",
            tokenizer_language="de",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

TRG = Field(tokenize = "spacy",
            tokenizer_language="en",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'),
                                                    fields = (SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 810kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 244kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 237kB/s]


In [0]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [0]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 4
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

In [0]:
import random
from typing import Tuple
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

In [0]:
class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src: Tensor) -> Tuple[Tensor]:
        print("\n\t--- ENCODER ---\n")
        print("Parameters-")
        print("src: {}".format(src.shape))

        embedded = self.dropout(self.embedding(src))
        
        print("embedded:", embedded.shape)
        
        outputs, hidden = self.rnn(embedded)

        print("\noutput values:", outputs)
        print("\nhidden values:", torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        
        print("outputs:", outputs.shape)
        print("hidden:", hidden.shape)
        
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        print("\nhidden mod values:", hidden)
        print("hidden mod:", hidden.shape)
        
        return outputs, hidden

In [0]:
class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.attn_in = (enc_hid_dim * 2) + dec_hid_dim
        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tensor:
        print("\n\t--- ATTENTION ---\n")
        print("Parameters-")
        print("decoder_hidden: {}, encoder_outputs: {}".format(decoder_hidden.shape, encoder_outputs.shape))
        
        src_len = encoder_outputs.shape[0]

        print("src_len:", src_len)

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

        print("repeated_decoder_hidden:", repeated_decoder_hidden.shape)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        print("encoder_outputs:", encoder_outputs.shape)

        energy = torch.tanh(self.attn(torch.cat((repeated_decoder_hidden, encoder_outputs), dim = 2)))

        print("energy:", energy.shape)

        attention = torch.sum(energy, dim=2)

        print("attention:", attention.shape)

        return F.softmax(attention, dim=1)

In [0]:
class Decoder(nn.Module):
    def __init__(self,
                 output_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: int,
                 attention: nn.Module):
        super().__init__()
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def _weighted_encoder_rep(self,
                              decoder_hidden: Tensor,
                              encoder_outputs: Tensor) -> Tensor:
        a = self.attention(decoder_hidden, encoder_outputs)
        
        print("  a:", a.shape)
        
        a = a.unsqueeze(1)
        
        print("  a unsqueezed:", a.shape)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        print("  encoder_outputs:", encoder_outputs.shape)
        
        weighted_encoder_rep = torch.bmm(a, encoder_outputs)

        print("  weighted_encoder_rep:", weighted_encoder_rep.shape)

        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)

        print("  weighted_encoder_rep:", weighted_encoder_rep.shape)

        return weighted_encoder_rep

    def forward(self,
                input: Tensor,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tuple[Tensor]:
        print("\n\t--- DECODER ---\n")
        print("Parameters-")
        print("input: {}, decoder_hidden: {}, encoder_outputs: {}".format(input.shape, decoder_hidden.shape, encoder_outputs.shape))

        input = input.unsqueeze(0)
        
        print("input:", input.shape)

        embedded = self.dropout(self.embedding(input))

        print("embedded:", embedded.shape)

        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden, encoder_outputs)

        print("weighted_encoder_rep:", weighted_encoder_rep.shape)

        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)

        print("rnn_input:", rnn_input.shape)

        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        print("output:", output.shape, ", decoder_hidden:", decoder_hidden.shape)

        embedded = embedded.squeeze(0)

        print("embedded squeezed:", embedded.shape)

        output = output.squeeze(0)

        print("output squeezed:", output.shape)

        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        print("weighted_encoder_rep squeezed:", weighted_encoder_rep.shape)

        output = self.out(torch.cat((output,
                                     weighted_encoder_rep,
                                     embedded), dim = 1))
        
        print("output (final):", output.shape)

        return output, decoder_hidden.squeeze(0)

In [0]:
class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                src: Tensor,
                trg: Tensor,
                teacher_forcing_ratio: float = 0.5) -> Tensor:
        print("\n\t--- SEQ2SEQ ---\n")
        print("Parameters-")
        print("src: {}, trg: {}".format(src.shape, trg.shape))
        
        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        print("outputs:", outputs.shape)
        
        encoder_outputs, hidden = self.encoder(src)
        
        print("encoder_outputs:", encoder_outputs.shape)
        print("hidden:", hidden.shape)

        # first input to the decoder is the <sos> token
        output = trg[0,:]
        
        print("output:", output.shape)

        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)
        return outputs

In [0]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
# ENC_EMB_DIM = 256
# DEC_EMB_DIM = 256
# ENC_HID_DIM = 512
# DEC_HID_DIM = 512
# ATTN_DIM = 64
# ENC_DROPOUT = 0.5
# DEC_DROPOUT = 0.5

# ENC_EMB_DIM = 32
# DEC_EMB_DIM = 32
# ENC_HID_DIM = 64
# DEC_HID_DIM = 64
# ATTN_DIM = 8

ENC_EMB_DIM = 2
DEC_EMB_DIM = 2
ENC_HID_DIM = 4
DEC_HID_DIM = 4
ATTN_DIM = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)
model = Seq2Seq(enc, dec, device).to(device)

def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

model.apply(init_weights)
optimizer = optim.Adam(model.parameters())

def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 116,337 trainable parameters


In [0]:
PAD_IDX = TRG.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [0]:
import math
import time

def train(model: nn.Module,
          iterator: BucketIterator,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):
    model.train()
    epoch_loss = 0
    for _, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        optimizer.zero_grad()
        output = model(src, trg)
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
        break
    return epoch_loss / len(iterator)


def evaluate(model: nn.Module,
             iterator: BucketIterator,
             criterion: nn.Module):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for _, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg
            output = model(src, trg, 0) #turn off teacher forcing
            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)


def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

N_EPOCHS = 1
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    # print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

# test_loss = evaluate(model, test_iterator, criterion)
# print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')


	--- SEQ2SEQ ---

Parameters-
src: torch.Size([14, 4]), trg: torch.Size([18, 4])
outputs: torch.Size([18, 4, 5893])

	--- ENCODER ---

Parameters-
src: torch.Size([14, 4])
embedded: torch.Size([14, 4, 2])

output values: tensor([[[-1.5934e-05, -9.0265e-07, -3.6109e-06,  2.2339e-05, -6.5083e-06,
           1.9935e-05,  2.3175e-05, -1.6225e-06],
         [-9.4182e-06, -2.9124e-06, -1.0047e-05,  3.1819e-05, -2.9814e-05,
          -1.5649e-06,  4.5519e-05,  4.3488e-06],
         [-1.5934e-05, -9.0265e-07, -3.6109e-06,  2.2339e-05, -3.8113e-06,
           9.8370e-06,  1.2330e-05, -1.2289e-06],
         [ 6.5154e-06, -2.0097e-06, -6.4361e-06,  9.4794e-06, -1.1918e-05,
          -1.5920e-05,  8.1693e-06,  3.7198e-06]],

        [[-8.0484e-06, -5.7864e-07, -1.7834e-06,  1.1132e-05, -6.8514e-06,
           1.7699e-05,  2.2251e-05, -9.4991e-07],
         [-1.4739e-05, -2.1741e-06, -7.2107e-06,  2.9758e-05, -3.9805e-05,
          -2.6696e-06,  6.0403e-05,  5.6155e-06],
         [-8.0484e-06, -5.